In [4]:
!ls /tmp

expected.npy		test_brevitas_fc_6u5_ans7
exported_qn_model.onnx	test_brevitas_fc_pvtpnyn7
finn_dev_mirza		test_brevitas_quartznet_k43b176w
partitioning_ivd8hw4i	test_brevitas_quartznet_kovnyhz0
partitioning_kfpdhp2y	test_streamline_cnv_5vp0w1lo
partitioning_lcfdzgnn	test_streamline_cnv_svvjo8no
produced.npy		test_streamline_fc_05ibpkj3
quartznet_export.onnx	test_streamline_fc_4kdx61fp


In [17]:
from finn.util.visualization import showInNetron

showInNetron("/tmp/quartznet_export.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/quartznet_export.onnx' at http://0.0.0.0:8081


In [14]:
from finn.util.visualization import showInNetron

showInNetron("/tmp/exported_qn_model.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/exported_qn_model.onnx' at http://0.0.0.0:8081


In [15]:
import numpy as np
from finn.core.modelwrapper import ModelWrapper
import finn.core.onnx_exec as oxe

model = ModelWrapper("/tmp/quartznet_export.onnx")

inp_val = np.load("input_sample_float_0.npy")
input_dict = {model.graph.input[0].name: inp_val}
output_dict_export = oxe.execute_onnx(model, input_dict, return_full_exec_context=True)


In [16]:
import numpy as np
from finn.core.modelwrapper import ModelWrapper
import finn.core.onnx_exec as oxe

model = ModelWrapper("/tmp/exported_qn_model.onnx")

z = model.get_initializer(model.graph.node[0].input[1])
s = model.get_initializer(model.graph.node[1].input[1])

inp_val = np.load("input_sample_float_0.npy")
inp_val = np.round(inp_val / s + z)
input_dict = {model.graph.input[0].name: inp_val}
output_dict_quantized = oxe.execute_onnx(model, input_dict, return_full_exec_context=True)



In [21]:
model_export = ModelWrapper("/tmp/quartznet_export.onnx")
export_out = output_dict_export[model_export.graph.output[0].name]

model_quantized = ModelWrapper("/tmp/exported_qn_model.onnx")
quantized_out = output_dict_quantized[model_quantized.graph.output[0].name]

for idx, el in enumerate(export_out.flatten()):
    print("{} - {} = {}".format(el, quantized_out.flatten()[idx], el-quantized_out.flatten()[idx]))

28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
22 - 22 = 0
28 - 28 = 0
28 - 28 = 0
1 - 1 = 0
28 - 28 = 0
28 - 28 = 0
14 - 14 = 0
28 - 28 = 0
28 - 28 = 0
9 - 9 = 0
28 - 28 = 0
28 - 28 = 0
20 - 20 = 0
28 - 28 = 0
28 - 28 = 0
25 - 25 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
0 - 0 = 0
0 - 0 = 0
28 - 28 = 0
1 - 1 = 0
14 - 14 = 0
4 - 4 = 0
4 - 28 = -24
0 - 0 = 0
0 - 0 = 0
28 - 28 = 0
22 - 22 = 0
28 - 28 = 0
28 - 28 = 0
5 - 5 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
24 - 24 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
1 - 1 = 0
28 - 28 = 0
28 - 28 = 0
28 - 28 = 0
20 - 20 = 0
9 - 9 = 0
9 - 9 = 0
28 - 28 = 0
15 - 15 = 0
14 - 14 = 0
28 - 28 = 0
28 - 28 = 0
0 - 0 = 0
0 - 0 = 0
0 - 0 = 0
28 - 28 = 0
28 - 28 = 0
15 - 15 = 0
6 - 22 = -16
28 - 28 

In [22]:
labels = [" ", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m",
         "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "'"]
    
def __ctc_decoder_predictions_tensor(tensor, labels):
    """
    Decodes a sequence of labels to words
    """
    blank_id = len(labels)
    hypotheses = []
    labels_map = dict([(i, labels[i]) for i in range(len(labels))])
    decoded_prediction = []
    previous = len(labels)  # id of a blank symbol
    prediction = tensor
    for p in prediction:
        if (p != previous or previous == blank_id) and p != blank_id:
            decoded_prediction.append(p)
        previous = p
    hypothesis = ''.join([labels_map[c] for c in decoded_prediction])
    hypotheses.append(hypothesis)
    return hypotheses

print(__ctc_decoder_predictions_tensor(export_out.flatten(),labels))
print(__ctc_decoder_predictions_tensor(quantized_out.flatten(),labels))

['vanity and vexation of spad']
['vanity and vexation ov span']
